In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

Train_path = 'dataset/train.csv'
Test_path = 'dataset/test.csv'

Data_train = pd.read_csv(Train_path)
Data_test = pd.read_csv(Test_path)
print("Setup complete.")

Setup complete.


In [10]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler,StandardScaler,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Imputer for categorical value ! 

# Inspired from stackoverflow.com/questions/25239958
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

In [56]:
#Numerical attributes

Numerical_attribs = ['Age','SibSp', 'Parch','Fare']

new_num_pipiline = Pipeline([
    ("fill_miss_value",SimpleImputer(strategy='median')),
    ("scaled",StandardScaler())                         
                             ])
Categorical_attribs = ['Pclass','Sex','Embarked']

new_cat_pipeline = Pipeline([
    ("fill_miss_value",MostFrequentImputer()),
    ("encoder",OneHotEncoder(sparse=False)),
                            ])

full_data_transformer = ColumnTransformer([
    ("numerical",new_num_pipiline,Numerical_attribs),
    ("categorical",new_cat_pipeline,Categorical_attribs)
                                          ])


Data_train = pd.read_csv(Train_path)
y_survived = Data_train.Survived.copy()
Data_train.drop('Survived',axis=1,inplace=True)
full_data_transformer.fit_transform(Data_train)

NameError: name 'StandardScaler' is not defined

In [57]:
# If we want to get the different columns

# Get the columns for the one hot encoder
cat_features = []

full_cat_list = full_data_transformer.named_transformers_['categorical']['encoder'].categories_

for cat_list in full_cat_list:
    for elt in cat_list:
        cat_features.append(elt)
        
Data_prepared = full_data_transformer.fit_transform(Data_train)
Features_list = Numerical_attribs + cat_features

# We can rebuild a pandas DataFrame
pd.DataFrame(Data_prepared,columns=Features_list)


,Age,SibSp,Parch,Fare,1,2,3,female,male,C,Q,S
0,0.271174,0.125,0.000000,0.014151,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,0.472229,0.125,0.000000,0.139136,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.321438,0.000,0.000000,0.015469,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,0.434531,0.125,0.000000,0.103644,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.434531,0.000,0.000000,0.015713,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.334004,0.000,0.000000,0.025374,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
887,0.233476,0.000,0.000000,0.058556,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
888,0.346569,0.125,0.333333,0.045771,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
889,0.321438,0.000,0.000000,0.058556,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


## Find a model 

### Random Forest Classifier

In [58]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_clf.fit(Data_prepared,y_survived)
forest_scores = cross_val_score(forest_clf, Data_prepared, y_survived, cv=10)

forest_scores.mean()

0.8126466916354558

#### Fine Tune the model

In [59]:
from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform

param_grid_forest = [ {'n_estimators':[10,50,100,200,300], 
                       'criterion':['gini','entropy'],'bootstrap':[True,False]},
                  ]

forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)

grid_search_forest = GridSearchCV(forest_clf,param_grid=param_grid_forest,
                               cv=5, scoring='accuracy',
                               verbose=3)

In [60]:
grid_search_forest.fit(Data_prepared,y_survived)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] bootstrap=True, criterion=gini, n_estimators=10 .................
[CV]  bootstrap=True, criterion=gini, n_estimators=10, score=0.754, total=   0.0s
[CV] bootstrap=True, criterion=gini, n_estimators=10 .................
[CV]  bootstrap=True, criterion=gini, n_estimators=10, score=0.787, total=   0.0s
[CV] bootstrap=True, criterion=gini, n_estimators=10 .................
[CV]  bootstrap=True, criterion=gini, n_estimators=10, score=0.837, total=   0.0s
[CV] bootstrap=True, criterion=gini, n_estimators=10 .................
[CV]  bootstrap=True, criterion=gini, n_estimators=10, score=0.798, total=   0.0s
[CV] bootstrap=True, criterion=gini, n_estimators=10 .................
[CV]  bootstrap=True, criterion=gini, n_estimators=10, score=0.820, total=   0.0s
[CV] bootstrap=True, criterion=gini, n_estimators=50 .................
[CV]  bootstrap=True, criterion=gini, n_estimators=50, score=0.760, total=   0.1s
[CV] bootstrap=True,

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  bootstrap=True, criterion=gini, n_estimators=50, score=0.798, total=   0.1s
[CV] bootstrap=True, criterion=gini, n_estimators=50 .................
[CV]  bootstrap=True, criterion=gini, n_estimators=50, score=0.854, total=   0.1s
[CV] bootstrap=True, criterion=gini, n_estimators=50 .................
[CV]  bootstrap=True, criterion=gini, n_estimators=50, score=0.781, total=   0.1s
[CV] bootstrap=True, criterion=gini, n_estimators=50 .................
[CV]  bootstrap=True, criterion=gini, n_estimators=50, score=0.831, total=   0.1s
[CV] bootstrap=True, criterion=gini, n_estimators=100 ................
[CV]  bootstrap=True, criterion=gini, n_estimators=100, score=0.771, total=   0.2s
[CV] bootstrap=True, criterion=gini, n_estimators=100 ................
[CV]  bootstrap=True, criterion=gini, n_estimators=100, score=0.792, total=   0.2s
[CV] bootstrap=True, criterion=gini, n_estimators=100 ................
[CV]  bootstrap=True, criterion=gini, n_estimators=100, score=0.860, total=   0.

[CV]  bootstrap=False, criterion=gini, n_estimators=50, score=0.820, total=   0.1s
[CV] bootstrap=False, criterion=gini, n_estimators=100 ...............
[CV]  bootstrap=False, criterion=gini, n_estimators=100, score=0.788, total=   0.2s
[CV] bootstrap=False, criterion=gini, n_estimators=100 ...............
[CV]  bootstrap=False, criterion=gini, n_estimators=100, score=0.781, total=   0.2s
[CV] bootstrap=False, criterion=gini, n_estimators=100 ...............
[CV]  bootstrap=False, criterion=gini, n_estimators=100, score=0.843, total=   0.2s
[CV] bootstrap=False, criterion=gini, n_estimators=100 ...............
[CV]  bootstrap=False, criterion=gini, n_estimators=100, score=0.775, total=   0.2s
[CV] bootstrap=False, criterion=gini, n_estimators=100 ...............
[CV]  bootstrap=False, criterion=gini, n_estimators=100, score=0.826, total=   0.2s
[CV] bootstrap=False, criterion=gini, n_estimators=200 ...............
[CV]  bootstrap=False, criterion=gini, n_estimators=200, score=0.788, t

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   22.6s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=42,
                                  

In [61]:
grid_search_forest.best_score_

0.809252400979223

In [62]:
for mean,params in zip(grid_search_forest.cv_results_['mean_test_score'],grid_search_forest.cv_results_['params']): 
    print(mean,"\n", params)

0.7991525955683887 
 {'bootstrap': True, 'criterion': 'gini', 'n_estimators': 10}
0.8047642960266147 
 {'bootstrap': True, 'criterion': 'gini', 'n_estimators': 50}
0.8036281463812692 
 {'bootstrap': True, 'criterion': 'gini', 'n_estimators': 100}
0.8047517418868871 
 {'bootstrap': True, 'criterion': 'gini', 'n_estimators': 200}
0.8047517418868871 
 {'bootstrap': True, 'criterion': 'gini', 'n_estimators': 300}
0.7935220639005711 
 {'bootstrap': True, 'criterion': 'entropy', 'n_estimators': 10}
0.805881614462369 
 {'bootstrap': True, 'criterion': 'entropy', 'n_estimators': 50}
0.8047454648170234 
 {'bootstrap': True, 'criterion': 'entropy', 'n_estimators': 100}
0.805881614462369 
 {'bootstrap': True, 'criterion': 'entropy', 'n_estimators': 200}
0.809252400979223 
 {'bootstrap': True, 'criterion': 'entropy', 'n_estimators': 300}
0.80248571966606 
 {'bootstrap': False, 'criterion': 'gini', 'n_estimators': 10}
0.7968740192078336 
 {'bootstrap': False, 'criterion': 'gini', 'n_estimators': 50

## Result 

In [65]:
final_model = grid_search_forest.best_estimator_

y_test_pred = final_model.predict(full_data_transformer.transform(Data_test))

#output = pd.DataFrame({'PassengerId': Data_test.PassengerId, 'Survived': y_test_pred})

#output.to_csv('dataset/submission.csv', index=False)

#!kaggle competitions submit -c titanic -f dataset/submission.csv -m "With rounded values (2nd)"

Successfully submitted to Titanic: Machine Learning from Disaster



  0%|          | 0.00/3.18k [00:00<?, ?B/s]
100%|##########| 3.18k/3.18k [00:03<00:00, 965B/s]
